### Use case 

Create a Customer service pre sales chatbot for a fictional  ecommerce company .

Users will come to the website and ask questions about order status.

The bot will ask user to enter either the order number or customer name.

Based on that the bot will fetch the order and item details from a sql database.

It will then respond to the customer about status of each item in the order.



### order information Database

We will create a sql database with 2 tables order_data and item_data.

To make it interesting, We will also use the LLM to do nlp/english language commands to sql conversion



### Libraries used 

gradio to create Chatbot UIs

langchain.

sqllite to store order information.

open AI for  nlp/english language commands to sql conversion

open AI for chat responses



### Install Gradio

In [ ]:
#!pip install --upgrade gradio

### Install Langchain

In [ ]:
#!pip install langchain==0.1.0
#!pip install langchain_openai
#!pip install openai==1.7.1


### Create order Information Database

In [1]:
# Import the sqlite3 library
import sqlite3

conn = sqlite3.connect('order_information.db',check_same_thread=False)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS order_data')

cursor.execute('''
CREATE TABLE order_data (
    order_no INTEGER PRIMARY KEY,
    customer_name TEXT
    )
''')

# Insert rows of data into the table
rows_to_insert = [
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie')
]

# Use executemany() to insert multiple rows
cursor.executemany('INSERT INTO order_data VALUES (?, ?)', rows_to_insert)

# Commit the changes
conn.commit()



In [2]:

cursor.execute('DROP TABLE IF EXISTS item_data')


cursor.execute('''
CREATE  TABLE item_data (
    order_no INTEGER ,
    item_no INTEGER,
    item_name TEXT,
    item_amount float,
    status TEXT
    )
''')

# Insert rows of data into the table
rows_to_insert = [
    (1, 100,'TV',600, 'Shipped'),
    (1, 101,'Fridge',200, 'In Process'),
    (2, 200,'VR glasses',400, 'Shipped'),
    (2, 201,'Pet robot',100, 'Shipped'),
    (3, 100,'TV',100, 'Cancelled')
]

# Use executemany() to insert multiple rows
cursor.executemany('INSERT INTO item_data VALUES (?, ?, ?, ?, ?)', rows_to_insert)

# Commit the changes
conn.commit()

# Always close the connection when done
#conn.close()


### NLP to SQL Conversion


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os

chat = ChatOpenAI(openai_api_key=os.getenv("open_ai_secret_key"))

database_definitions= """ sql tables with table_names and column names as below.
order_data (order_no, cusotmer_name)
item_data (order_no, item_no, item_name, item_amount, status)

"""

order_number_query = f"""create a sql query to find order number, customer name, item_name, item number, 
amount, status using the table defintions specified in {database_definitions}.
accept input_order_number as input in the where clause. It should be enclosed in square brackets.
Just return the sql query as a string, do not add any explanation.
Do not include ticks or word sql in the ouput.
"""


#print ('query',query)

messages = [
    SystemMessage(content="you are a helpful assistant "),
    HumanMessage(content=order_number_query),
]
resp= chat.invoke(messages)

print ('resp',resp.content)

resp SELECT od.order_no, od.customer_name, id.item_name, id.item_no, id.item_amount, id.status
FROM order_data od
JOIN item_data id ON od.order_no = id.order_no
WHERE id.order_no = [input_order_number];


In [55]:
sql_query = resp.content.replace('[input_order_number]','1')
sql_query

'SELECT order_data.order_no, order_data.customer_name, item_data.item_name, item_data.item_no, item_data.item_amount, item_data.status\nFROM order_data\nJOIN item_data ON order_data.order_no = item_data.order_no\nWHERE order_data.order_no = 1;'

In [56]:
import pandas as pd

df = pd.read_sql_query(sql_query, conn)
df


,order_no,customer_name,item_name,item_no,item_amount,status
0,1,Alice,TV,100,600.0,Shipped
1,1,Alice,Fridge,101,200.0,In Process


In [57]:
customer_name_query = f"""create a sql query to find order number, customer name, item_name, item number, 
amount, status using the table defintions specified in {database_definitions}.
accept input_customer_name as input in the where clause.It should be enclosed in square brackets.

Just return the sql query as a string, do not add any explanation.
Do not include ticks or word sql in the ouput.
"""


#print ('query',query)

messages = [
    SystemMessage(content="you are a helpful assistant "),
    HumanMessage(content=customer_name_query),
]
resp= chat.invoke(messages)

print ('resp',resp.content)

sql_query = resp.content.replace('[input_customer_name]',"'Bob'")
sql_query
import pandas as pd

df = pd.read_sql_query(sql_query, conn)
df

resp SELECT od.order_no, od.customer_name, it.item_name, it.item_no, it.item_amount, it.status
FROM order_data od
JOIN item_data it ON od.order_no = it.order_no
WHERE od.customer_name = [input_customer_name];


,order_no,customer_name,item_name,item_no,item_amount,status
0,2,Bob,VR glasses,200,400.0,Shipped
1,2,Bob,Pet robot,201,100.0,Shipped


In [4]:
order_number_or_customer_name_query = f"""create a sql query to find order number, customer name, item_name, item number, 
amount, status using the table defintions specified in {database_definitions}.
accept input_customer_name as input in the where clause.It should be enclosed in square brackets.
accept input_order_number as input in the where clause. It should be enclosed in square brackets.
use or clause to include both these inputs.
Just return the sql query as a string, do not add any explanation.
Do not include ticks or word sql in the ouput.
"""


#print ('query',query)

messages = [
    SystemMessage(content="you are a helpful assistant "),
    HumanMessage(content=order_number_or_customer_name_query),
]
resp= chat.invoke(messages)

print ('resp',resp.content)



resp SELECT od.order_no, od.customer_name, id.item_name, id.item_no, id.item_amount, id.status
FROM order_data od
JOIN item_data id ON od.order_no = id.order_no
WHERE od.customer_name = [input_customer_name] OR id.order_no = [input_order_number];


In [23]:
sql_query = resp.content.replace('[input_customer_name]',"'Bob'")
sql_query = sql_query.replace('[input_order_number]','-999')
import pandas as pd

df = pd.read_sql_query(sql_query, conn)
df

,order_no,customer_name,item_name,item_no,item_amount,status
0,2,Bob,VR glasses,200,400.0,Shipped
1,2,Bob,Pet robot,201,100.0,Shipped


In [64]:
sql_query = resp.content.replace('[input_customer_name]',"'unknown'")
sql_query = sql_query.replace('[input_order_number]','2')
import pandas as pd

df = pd.read_sql_query(sql_query, conn)
df

,order_no,customer_name,item_name,item_no,item_amount,status
0,2,Bob,VR glasses,200,400.0,Shipped
1,2,Bob,Pet robot,201,100.0,Shipped


### Define function to get order status from database

In [5]:
import pandas as pd

def get_order_info(inp, inp_type):
    if inp_type == 'name':
        sql_query = resp.content.replace('[input_customer_name]',f"'{inp}'")        
        sql_query = sql_query.replace('[input_order_number]','-999')
    else:
        sql_query = resp.content.replace('[input_customer_name]',"'unknown'")
        sql_query = sql_query.replace('[input_order_number]',f"{inp}")       
    df = pd.read_sql_query(sql_query, conn)
    customer_name = df['customer_name'].unique()
    cust_status= f'Dear {customer_name[0]}, As per our tracking information here are the status of items in your order\n'
    for i,r in df.iterrows():
        item_name = r['item_name']
        status = r['status']
        cust_status += f"{item_name} is in status {status}\n"
    return cust_status

In [67]:
inp = 'Bob'
inp_type = 'name'
get_order_info(inp, inp_type)

cust_status Dear Bob, As per our tracking information here are the status of items in your order
VR glasses is in status Shipped
Pet robot is in status Shipped



'Dear Bob, As per our tracking information here are the status of items in your order\nVR glasses is in status Shipped\nPet robot is in status Shipped\n'

In [ ]:
# !pip install spacy

In [ ]:
#!python -m spacy download en_core_web_sm

## Create Chatbot Code 

In [7]:
import gradio as gr
import os
import spacy
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import pandas as pd
#from .autonotebook import tqdm as notebook_tqdm

chat = ChatOpenAI(openai_api_key=os.getenv("open_ai_secret_key"))
nlp = spacy.load("en_core_web_sm")

def chat_with_customer(message, history):
   
    chat_history = []
    inp_type=''
    augmented_query= ''

    try:
        # Attempt to convert the input to an integer
        converted_input = int(message)
        print(f"order_no,{converted_input}")
        inp_type = 'order_no'
        qry_result = get_order_info(converted_input, inp_type)
        print ('qry_result',qry_result)
        augmented_query=qry_result
    except ValueError:
        # If a ValueError occurs, it means the input wasn't an integer
        print(f"The input '{message}' is not an integer.")

    if inp_type != 'order_no':
        doc = nlp(message)
        # Look for proper nouns that could be names
        name = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
        if len(name) !=0 :
            inp_type = 'name'
            print('name',name)
            qry_result = get_order_info(name[0], inp_type)
            print ('qry_result',qry_result)
            augmented_query=qry_result
    
    if 'order' in message:
        augmented_query = """ask customer to provide order number or customer name. 
        
        """
    elif augmented_query != '':
         augmented_query = "answer using your name as ai chatbot and display the below message " + augmented_query
    else:
        augmented_query=message   
    print ('augmented_query',augmented_query)
    chat_history.append(HumanMessage(content=augmented_query))
    llm_response = chat.invoke(chat_history).content
    return llm_response
    
gr.ChatInterface(chat_with_customer).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


### Sample output when customer enters order number

![order_sample](order_sample.png)

### Sample output when customer enters name

![name_sample](name_sample.png)